In [0]:
### This is our topic where messages are dropped 
messages_path = "/tmp/message_system/messages"

##this where the spark structured sreaming stores the progress info
checkpoint_path = "/tmp/messaging_system/checkpoint"

##clean up previous runs if needed 

dbutils.fs.rm(messages_path,recurse = True)
dbutils.fs.rm(checkpoint_path,recurse = True)

##create main directory
dbutils.fs.mkdirs(messages_path)

print(f"Directory '{messages_path}' is ready for messages.")

Directory '/tmp/message_system/messages' is ready for messages.


In [0]:
from pyspark.sql.types import StructType, StructField , StringType, DoubleType , TimestampType
from pyspark.sql.functions import *


schema = StructType([
    StructField("device_id",StringType(),True),
    StructField("temperature",DoubleType(),True),
    StructField("location",StringType(),True),
    StructField("event_time",TimestampType(),True)
])


##reading the files 
raw_stream_df = spark \
    .readStream \
        .schema(schema) \
            .json(messages_path)


#transformation of adding new column
processed_stream_df = raw_stream_df.withColumn("processing_staus", lit("new"))

##writing the df into in memory table so we can query the data
query = processed_stream_df \
    .writeStream \
        .format("memory") \
            .queryName("message_queue") \
                .outputMode("append") \
                    .start()


print("Streaming consumer is now running. waiting for messages...")




Streaming consumer is now running. waiting for messages...


In [0]:
%sql

select * from message_queue order by event_time desc

device_id,temperature,location,event_time,processing_staus
device_1,23.57,server_room_a,2025-07-03T14:42:54.442+0000,new
device_4,20.88,office_floor_b,2025-07-03T14:41:53.241+0000,new
device_8,17.36,office_floor_b,2025-07-03T14:40:48.079+0000,new
device_4,23.37,office_floor_b,2025-07-03T14:40:33.915+0000,new
device_2,27.41,server_room_a,2025-07-03T14:39:34.733+0000,new
device_4,16.99,office_floor_b,2025-07-03T14:38:27.940+0000,new


In [0]:
json_read  = spark.read.json("dbfs:/tmp/message_system/messages")

In [0]:
json_read.show()

+---------+--------------------+--------------+-----------+
|device_id|          event_time|      location|temperature|
+---------+--------------------+--------------+-----------+
| device_4|2025-07-03T14:38:...|office_floor_b|      16.99|
| device_4|2025-07-03T14:40:...|office_floor_b|      23.37|
| device_8|2025-07-03T14:40:...|office_floor_b|      17.36|
| device_4|2025-07-03T14:41:...|office_floor_b|      20.88|
| device_2|2025-07-03T14:39:...| server_room_a|      27.41|
| device_1|2025-07-03T14:42:...| server_room_a|      23.57|
+---------+--------------------+--------------+-----------+

